# Notebook 03: Embeddings e Ingestão no Pinecone

Este notebook realiza a ingestão dos dados processados no Pinecone. Aqui vamos:

1. **Conectar** com o índice Pinecone (biobyia)
2. **Configurar** o gerenciador de embeddings (Gemini ou Ollama)
3. **Gerar** embeddings para todos os chunks
4. **Ingerir** dados no Pinecone em lotes
5. **Verificar** ingestão bem-sucedida

## 📋 Pré-requisitos

- Notebook 02 executado com sucesso (chunks processados)
- Variáveis de ambiente configuradas (PINECONE_API_KEY, GEMINI_API_KEY, etc.)
- Índice Pinecone criado (biobyia)

## ⚠️ IMPORTANTE

- Este processo pode levar vários minutos dependendo do tamanho do dataset
- Certifique-se de ter créditos suficientes no Pinecone
- A ingestão é feita em lotes para otimizar performance

## 🔄 Ordem de Execução

Execute as células **sequencialmente** (de cima para baixo).


In [2]:
%pip install python-dotenv tenacity pinecone langchain langchain-community langchain-google-genai google-generativeai tiktoken sentence-transformers tqdm

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install python-dotenv tenacity pinecone langchain langchain-community langchain-google-genai google-generativeai tiktoken sentence-transformers tqdm

In [8]:
# ============================================================================
# ETAPA 0: IMPORTAÇÃO DE BIBLIOTECAS E CONFIGURAÇÕES
# ============================================================================
# Esta célula importa todas as bibliotecas necessárias e configura o ambiente
# Execute esta célula PRIMEIRO antes de qualquer outra operação

import sys
import importlib
import os
from pathlib import Path

# Adiciona o diretório raiz ao path para importar módulos
# Quando executado do diretório notebooks/, o parent é rag_medical/
root_dir = Path.cwd().parent
if str(root_dir) not in sys.path:
    sys.path.insert(0, str(root_dir))

# ============================================================================
# CARREGA VARIÁVEIS DE AMBIENTE DO ARQUIVO .env
# ============================================================================
# Garante que o .env seja carregado do diretório raiz do projeto (rag_medical/)
try:
    from dotenv import load_dotenv
    
    # Caminho para o arquivo .env no diretório raiz do projeto
    env_path = root_dir / '.env'
    
    if env_path.exists():
        load_dotenv(dotenv_path=str(env_path))
        print(f"✅ Arquivo .env carregado: {env_path}")
    else:
        print(f"⚠️  Arquivo .env não encontrado em: {env_path}")
        print(f"   Crie o arquivo .env baseado em env.example")
        print(f"   Localização esperada: {env_path}")
        
except ImportError:
    print("⚠️  python-dotenv não instalado. Variáveis de ambiente do sistema serão usadas.")
    print("   Instale com: pip install python-dotenv")

# Recarrega módulos se já foram importados (útil durante desenvolvimento)
# IMPORTANTE: Recarrega na ordem correta (dependências primeiro)
modules_to_reload = [
    'config.settings',
    'config',
    'scripts.embeddings_manager',
    'scripts.pinecone_ingester'
]

for module_name in modules_to_reload:
    if module_name in sys.modules:
        try:
            importlib.reload(sys.modules[module_name])
            print(f"🔄 Módulo recarregado: {module_name}")
        except Exception as e:
            print(f"⚠️  Aviso ao recarregar {module_name}: {e}")

# Importa configurações
from config.settings import get_settings

# Carrega configurações
settings = get_settings()

# Tenta importar módulos do pipeline RAG (podem falhar se dependências não estiverem instaladas)
try:
    from scripts.embeddings_manager import EmbeddingsManager
except ImportError as e:
    print(f"❌ Erro ao importar EmbeddingsManager: {e}")
    print(f"   Instale as dependências no ambiente virtual:")
    print(f"   cd {root_dir}")
    print(f"   pip install -r requirements.txt")
    EmbeddingsManager = None

try:
    from scripts.pinecone_ingester import PineconeIngester
except ImportError as e:
    print(f"❌ Erro ao importar PineconeIngester: {e}")
    print(f"   Instale as dependências no ambiente virtual:")
    print(f"   cd {root_dir}")
    print(f"   pip install -r requirements.txt")
    print(f"   Especificamente: pip install tenacity pinecone")
    PineconeIngester = None

# Verifica se os módulos foram importados com sucesso ANTES de validar configurações
if EmbeddingsManager is None or PineconeIngester is None:
    print("\n" + "=" * 80)
    print("❌ MÓDULOS NECESSÁRIOS NÃO FORAM IMPORTADOS!")
    print("=" * 80)
    print(f"📦 Instale as dependências no ambiente virtual:")
    print(f"   1. Ative o ambiente virtual")
    print(f"   2. cd {root_dir}")
    print(f"   3. pip install -r requirements.txt")
    print("=" * 80)
    raise ImportError("Dependências não instaladas")

# Valida configurações (strict=True porque este notebook precisa de Pinecone e Embeddings)
is_valid, errors = settings.validate(strict=True)
if not is_valid:
    print("\n" + "=" * 80)
    print("❌ ERROS DE CONFIGURAÇÃO:")
    print("=" * 80)
    for error in errors:
        print(f"   - {error}")
    print("\n💡 SOLUÇÃO:")
    print(f"   1. Crie o arquivo .env em: {root_dir / '.env'}")
    print(f"   2. Copie de: {root_dir / 'env.example'}")
    print(f"   3. Preencha com suas credenciais reais")
    print("=" * 80)
    raise ValueError("Configurações inválidas")

print("\n✅ Bibliotecas importadas com sucesso!")
settings.print_config()


✅ Arquivo .env carregado: /Users/vitorteixeira/Developer/projects/tech_challenge_fase_3_v2/rag_medical/.env
🔄 Módulo recarregado: config.settings
🔄 Módulo recarregado: config
🔄 Módulo recarregado: scripts.embeddings_manager
🔄 Módulo recarregado: scripts.pinecone_ingester

✅ Bibliotecas importadas com sucesso!
⚙️  CONFIGURAÇÃO DO PIPELINE RAG
Pinecone Index: biobyia
Pinecone Namespace: medical_qa
Embedding Provider: gemini
Embedding Model: text-embedding-004
Data Path: /Users/vitorteixeira/Developer/projects/tech_challenge_fase_3_v2/rag_medical/ori_pqal.json
Chunk Size: 512
Chunk Overlap: 50
Batch Size: 100
Top K Results: 5


In [9]:
# ============================================================================
# ETAPA 1: VERIFICAÇÃO E CARREGAMENTO DE CHUNKS
# ============================================================================
# Tenta carregar chunks de múltiplas fontes:
# 1. Variável all_chunks na memória (se notebook 02 foi executado)
# 2. Arquivo JSON salvo (se chunks foram salvos anteriormente)
# 3. Reprocessamento dos dados (se necessário)

import json

# Verifica se all_chunks já existe na memória
try:
    chunks_count = len(all_chunks)
    print(f"✅ Chunks já carregados na memória: {chunks_count} chunks prontos para ingestão")
except NameError:
    print("⚠️  Variável 'all_chunks' não encontrada na memória")
    print("   Tentando carregar de arquivo salvo...")
    
    # Tenta carregar de arquivo JSON salvo
    chunks_loaded = False
    possible_chunk_files = [
        root_dir / 'processed_chunks.json',
        root_dir / 'chunks.json',
        root_dir / 'data' / 'processed_chunks.json',
    ]
    
    for chunk_file in possible_chunk_files:
        if chunk_file.exists():
            try:
                print(f"   📂 Tentando carregar: {chunk_file}")
                with open(chunk_file, 'r', encoding='utf-8') as f:
                    all_chunks = json.load(f)
                chunks_count = len(all_chunks)
                print(f"✅ Chunks carregados do arquivo: {chunks_count} chunks")
                chunks_loaded = True
                break
            except Exception as e:
                print(f"   ⚠️  Erro ao carregar {chunk_file}: {e}")
                continue
    
    # Se não conseguiu carregar de arquivo, exibe instruções
    if not chunks_loaded:
        chunk_file_path = root_dir / 'processed_chunks.json'
        print("\n" + "=" * 80)
        print("⚠️  CHUNKS NÃO ENCONTRADOS")
        print("=" * 80)
        print("A variável 'all_chunks' não está disponível.")
        print("\n💡 SOLUÇÕES:")
        print("\n📝 Opção 1: Execute o notebook 02 completo")
        print("   1. Abra o notebook 02-process-medical-data.ipynb")
        print("   2. Execute TODAS as células sequencialmente:")
        print("      - Célula 1: Importação de bibliotecas")
        print("      - Célula 2: Carregamento do dataset")
        print("      - Célula 3: Processamento de dados")
        print("      - Célula 4: Filtragem de entradas")
        print("      - Célula 5: Divisão em chunks (cria 'all_chunks')")
        print("   3. Depois, volte para este notebook e execute esta célula novamente")
        print("\n💾 Opção 2: Carregue de um arquivo JSON salvo")
        print("   Se você já processou os dados antes, salve os chunks:")
        print("   (Execute no notebook 02, após a célula 5)")
        print("   import json")
        print(f"   with open('{chunk_file_path}', 'w') as f:")
        print("       json.dump(all_chunks, f, ensure_ascii=False, indent=2)")
        print("\n   Depois, coloque o arquivo em um destes locais:")
        for chunk_file in possible_chunk_files:
            print(f"   - {chunk_file}")
        print("=" * 80)
        raise NameError(
            "Variável 'all_chunks' não encontrada. "
            "Execute o notebook 02 primeiro (especialmente a célula 5 que cria 'all_chunks') "
            "ou carregue os chunks de um arquivo JSON."
        )


✅ Chunks já carregados na memória: 5320 chunks prontos para ingestão


In [10]:
# ============================================================================
# ETAPA 2: INICIALIZAÇÃO DO GERENCIADOR DE EMBEDDINGS
# ============================================================================
# Configura o provider de embeddings (Gemini ou Ollama)
# Gemini é recomendado por ser mais rápido e confiável

print("=" * 80)
print("🔧 CONFIGURANDO EMBEDDINGS")
print("=" * 80)

# Cria gerenciador de embeddings
# Se não especificar provider, detecta automaticamente das configurações
embeddings_manager = EmbeddingsManager()

# Exibe informações sobre o provider
embedding_dim = embeddings_manager.get_embedding_dimension()
print(f"\nDimensão dos embeddings: {embedding_dim}")
print(f"Provider: {embeddings_manager.provider}")
print(f"Model: {embeddings_manager.model_name}")

print("=" * 80)


🔧 CONFIGURANDO EMBEDDINGS
✅ Embeddings Gemini inicializados: models/text-embedding-004

Dimensão dos embeddings: 768
Provider: gemini
Model: text-embedding-004


In [11]:
# ============================================================================
# ETAPA 3: INICIALIZAÇÃO DO INGESTER DO PINECONE
# ============================================================================
# Conecta com o índice Pinecone e prepara para ingestão
#
# ⚠️ IMPORTANTE: Se você modificou o código de pinecone_ingester.py,
# execute a célula 3 novamente para recarregar o módulo antes desta célula

print("=" * 80)
print("🔌 CONECTANDO COM PINECONE")
print("=" * 80)

# Recarrega o módulo para garantir que temos a versão mais recente
if 'scripts.pinecone_ingester' in sys.modules:
    importlib.reload(sys.modules['scripts.pinecone_ingester'])
    # Re-importa para garantir que temos a classe atualizada
    from scripts.pinecone_ingester import PineconeIngester

# Cria ingester do Pinecone
ingester = PineconeIngester(
    embeddings_manager=embeddings_manager,
    index_name=settings.PINECONE_INDEX_NAME,
    namespace=settings.PINECONE_NAMESPACE,
    api_key=settings.PINECONE_API_KEY
)

print("=" * 80)


🔌 CONECTANDO COM PINECONE
✅ Pinecone inicializado: índice 'biobyia'
   Namespace: medical_qa
   Dimensão dos embeddings: 768
   Dimensão do índice Pinecone: 768
   ✅ Dimensões compatíveis!


In [12]:
# ============================================================================
# ETAPA 4: INGESTÃO NO PINECONE
# ============================================================================
# Esta é a etapa principal: gera embeddings e ingere todos os chunks
# no Pinecone em lotes otimizados
#
# ⚠️ ATENÇÃO: Este processo pode levar vários minutos!
# - Para ~10.000 chunks: ~10-15 minutos
# - Para ~100.000 chunks: ~2-3 horas
#
# O processo é feito em lotes para:
# - Otimizar uso de API
# - Evitar rate limiting
# - Permitir retry em caso de erro
#
# 💾 CHECKPOINTING AUTOMÁTICO:
# - O processo salva automaticamente o progresso a cada 10 lotes
# - Se interrompido (Ctrl+C), você pode retomar executando esta célula novamente
# - O checkpoint é salvo em: rag_medical/checkpoints/
# - O processo continuará automaticamente de onde parou

import time

# Validação pré-ingestão
try:
    chunks_available = len(all_chunks) > 0
except NameError:
    print("=" * 80)
    print("❌ ERRO: Chunks não encontrados")
    print("=" * 80)
    print("A variável 'all_chunks' não está definida.")
    print("\n💡 SOLUÇÃO:")
    print("   1. Execute a célula 4 (ETAPA 1) primeiro para carregar os chunks")
    print("   2. Ou execute o notebook 02 para processar os dados")
    print("=" * 80)
    raise NameError(
        "Variável 'all_chunks' não encontrada. "
        "Execute a célula 4 (ETAPA 1) primeiro ou o notebook 02."
    )

if not chunks_available:
    raise ValueError("Nenhum chunk disponível para ingestão. Execute o notebook 02 primeiro.")

total_chunks = len(all_chunks)
batch_size = settings.BATCH_SIZE
total_batches = (total_chunks + batch_size - 1) // batch_size

# Estimativa de tempo (baseada em ~1-2 segundos por lote)
estimated_minutes = max(1, int(total_batches * 1.5 / 60))
if estimated_minutes < 60:
    estimated_time = f"~{estimated_minutes} min"
else:
    hours = estimated_minutes // 60
    mins = estimated_minutes % 60
    estimated_time = f"~{hours}h {mins}min" if mins > 0 else f"~{hours}h"

print("=" * 80)
print("🚀 INICIANDO INGESTÃO NO PINECONE")
print("=" * 80)
print(f"📊 Informações da Ingestão:")
print(f"   • Total de chunks: {total_chunks:,}")
print(f"   • Tamanho do lote: {batch_size}")
print(f"   • Total de lotes: {total_batches:,}")
print(f"   • Índice Pinecone: {settings.PINECONE_INDEX_NAME}")
if settings.PINECONE_NAMESPACE:
    print(f"   • Namespace: {settings.PINECONE_NAMESPACE}")
print(f"   • Provider de embeddings: {embeddings_manager.provider}")
print(f"   • Modelo: {embeddings_manager.model_name}")
print(f"\n⏱️  Tempo estimado: {estimated_time}")
print(f"⏳ Este processo pode levar vários minutos...")
print("=" * 80)

# Verifica se o módulo foi recarregado corretamente
# Se você receber erro sobre parâmetros inesperados, execute as células 3 e 6 novamente
import inspect
ingest_method = getattr(ingester, 'ingest_chunks', None)
if ingest_method:
    sig = inspect.signature(ingest_method)
    has_checkpoint_params = 'resume_from_checkpoint' in sig.parameters
    if not has_checkpoint_params:
        print("⚠️  AVISO: O módulo pinecone_ingester não foi recarregado!")
        print("   Execute as células 3 e 6 novamente para carregar a versão atualizada.")
        print("   Ou reinicie o kernel e execute todas as células novamente.")
        raise RuntimeError(
            "Módulo não atualizado. Execute as células 3 e 6 novamente, "
            "ou reinicie o kernel."
        )

# Registra tempo de início
start_time = time.time()

# Realiza a ingestão com suporte a checkpointing
# O checkpointing permite retomar de onde parou em caso de interrupção
try:
    ingestion_stats = ingester.ingest_chunks(
        all_chunks,
        batch_size=batch_size,
        show_progress=True,
        resume_from_checkpoint=True,  # Retoma de checkpoint se existir
        checkpoint_interval=10  # Salva checkpoint a cada 10 lotes
    )
    
    # Calcula tempo decorrido
    elapsed_time = time.time() - start_time
    elapsed_minutes = int(elapsed_time // 60)
    elapsed_seconds = int(elapsed_time % 60)
    
    print("\n" + "=" * 80)
    print("📊 ESTATÍSTICAS DA INGESTÃO")
    print("=" * 80)
    
    # Status da ingestão
    success_rate = (ingestion_stats['total_vectors'] / ingestion_stats['total_chunks'] * 100) if ingestion_stats['total_chunks'] > 0 else 0
    
    if ingestion_stats.get('interrupted', False):
        status = "⏸️  Interrompida (checkpoint salvo)"
    elif success_rate == 100:
        status = "✅ Concluída com sucesso"
    elif success_rate >= 95:
        status = "⚠️  Concluída com alguns erros"
    else:
        status = "❌ Concluída com muitos erros"
    
    print(f"{status}")
    print(f"\n📈 Resultados:")
    print(f"   • Total de chunks processados: {ingestion_stats['total_chunks']:,}")
    print(f"   • Total de vetores inseridos: {ingestion_stats['total_vectors']:,}")
    print(f"   • Total de lotes processados: {ingestion_stats['batches']:,}")
    print(f"   • Taxa de sucesso: {success_rate:.1f}%")
    print(f"\n⏱️  Tempo decorrido: {elapsed_minutes}min {elapsed_seconds}s")
    
    if ingestion_stats.get('interrupted', False):
        print(f"\n💾 Checkpoint salvo em: {ingestion_stats.get('checkpoint_path', 'N/A')}")
        print(f"   Para retomar, execute esta célula novamente.")
        print(f"   O processo continuará automaticamente de onde parou.")
    
    if ingestion_stats['errors']:
        print(f"\n⚠️  Erros encontrados: {len(ingestion_stats['errors'])}")
        print("   Primeiros erros:")
        for i, error in enumerate(ingestion_stats['errors'][:5], 1):
            print(f"   {i}. {error}")
        if len(ingestion_stats['errors']) > 5:
            print(f"   ... e mais {len(ingestion_stats['errors']) - 5} erros")
    else:
        print(f"\n✅ Nenhum erro encontrado!")
    
    print("=" * 80)
    
except KeyboardInterrupt:
    print("\n" + "=" * 80)
    print("⚠️  INGESTÃO INTERROMPIDA PELO USUÁRIO")
    print("=" * 80)
    print("A ingestão foi cancelada. Um checkpoint foi salvo automaticamente.")
    print("\n💡 Para retomar:")
    print("   1. Execute esta célula novamente")
    print("   2. O processo continuará automaticamente de onde parou")
    print("   3. Os chunks já processados foram inseridos no Pinecone")
    print("=" * 80)
    # Não re-raise para permitir que o usuário veja a mensagem
    ingestion_stats = {"interrupted": True, "total_vectors": 0, "total_chunks": total_chunks}
    
except Exception as e:
    print("\n" + "=" * 80)
    print("❌ ERRO DURANTE A INGESTÃO")
    print("=" * 80)
    print(f"Erro: {e}")
    print("\n💡 Possíveis soluções:")
    print("   1. Verifique sua conexão com a internet")
    print("   2. Verifique suas credenciais do Pinecone e Gemini")
    print("   3. Verifique se há créditos suficientes no Pinecone")
    print("   4. Tente executar novamente com um batch_size menor")
    print("   5. Se foi interrompido, execute novamente para retomar de checkpoint")
    print("=" * 80)
    raise


🚀 INICIANDO INGESTÃO NO PINECONE
📊 Informações da Ingestão:
   • Total de chunks: 5,320
   • Tamanho do lote: 100
   • Total de lotes: 54
   • Índice Pinecone: biobyia
   • Namespace: medical_qa
   • Provider de embeddings: gemini
   • Modelo: text-embedding-004

⏱️  Tempo estimado: ~1 min
⏳ Este processo pode levar vários minutos...

📋 Checkpoint encontrado! Retomando de índice 0
   Já processados: 0/5320 chunks

🚀 Iniciando ingestão de 5320 chunks no Pinecone...
   Batch size: 100
   Índice: biobyia
   Namespace: medical_qa


Ingerindo chunks:   0%|          | 0/5320 [00:00<?, ?it/s]

   Gerando embeddings para 100 chunks...


Ingerindo chunks:   0%|          | 1/5320 [00:04<6:56:32,  4.70s/it]

   Gerando embeddings para 100 chunks...


...

   Gerando embeddings para 100 chunks...


Ingerindo chunks:   1%|          | 53/5320 [02:57<4:28:32,  3.06s/it]

   Gerando embeddings para 20 chunks...


Ingerindo chunks:   1%|          | 54/5320 [02:58<4:50:34,  3.31s/it]


✅ Ingestão concluída!
   Vetores inseridos: 5320/5320

📊 ESTATÍSTICAS DA INGESTÃO
✅ Concluída com sucesso

📈 Resultados:
   • Total de chunks processados: 5,320
   • Total de vetores inseridos: 5,320
   • Total de lotes processados: 54
   • Taxa de sucesso: 100.0%

⏱️  Tempo decorrido: 2min 58s

✅ Nenhum erro encontrado!


In [13]:
# ============================================================================
# ETAPA 5: VERIFICAÇÃO DA INGESTÃO
# ============================================================================
# Verifica se os dados foram ingeridos corretamente fazendo uma query de teste

print("=" * 80)
print("🔍 VERIFICANDO INGESTÃO")
print("=" * 80)

# Importa função de query
from scripts.rag_query import query_medical_rag

# Faz uma query de teste
test_query = "mitochondria apoptosis"
print(f"Query de teste: '{test_query}'")
print("-" * 80)

try:
    results = query_medical_rag(
        test_query,
        embeddings_manager=embeddings_manager,
        top_k=3
    )
    
    print(f"✅ Query executada com sucesso!")
    print(f"   Resultados encontrados: {len(results)}")
    
    if results:
        print("\n📄 Primeiro resultado:")
        print(f"   Article ID: {results[0]['article_id']}")
        print(f"   Score: {results[0]['score']:.3f}")
        print(f"   Texto (primeiros 200 caracteres):")
        print(f"   {results[0]['text'][:200]}...")
    else:
        print("⚠️  Nenhum resultado encontrado. Verifique se a ingestão foi bem-sucedida.")
        
except Exception as e:
    print(f"❌ Erro ao executar query de teste: {e}")

print("=" * 80)


🔍 VERIFICANDO INGESTÃO
Query de teste: 'mitochondria apoptosis'
--------------------------------------------------------------------------------
✅ Query executada com sucesso!
   Resultados encontrados: 3

📄 Primeiro resultado:
   Article ID: 21645374
   Score: 0.600
   Texto (primeiros 200 caracteres):
   dye MitoTracker Red CMXRos and examined. Mitochondrial dynamics were delineated into four categories (M1-M4) based on characteristics including distribution, motility, and membrane potential (ΔΨm). A ...


## ✅ Conclusão da Etapa 3

Neste notebook você:
- ✅ Configurou o gerenciador de embeddings
- ✅ Conectou com o índice Pinecone
- ✅ Gerou embeddings para todos os chunks
- ✅ Ingeriu dados no Pinecone em lotes
- ✅ Verificou a ingestão bem-sucedida

## 📌 Próximos Passos

Agora você está pronto para o próximo notebook:
- **Notebook 04**: Testes de queries RAG
- Validação de recuperação de contexto
- Exemplos práticos de uso
